In [1]:
!pip install SpeechRecognition moviepy

  Using cached moviepy-1.0.3.tar.gz (388 kB)
  Using cached proglog-0.1.9.tar.gz (10 kB)
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110732 sha256=f287a75570cb3ca60545d0003eed84687b9044e2eaa0860005ec217dbcfdb254
  Stored in directory: c:\users\arjun janamatti\appdata\local\pip\cache\wheels\be\dc\17\8b4d5a63bcd05dc44db7da57e193372ccd333617293f9deebe
  Created wheel for proglog: filename=proglog-0.1.9-py3-none-any.whl size=6153 sha256=7fd619e3db2c3df131a7ffcc0a0b1d5554f8c31bc97020665ec4c1470f613886
  Stored in directory: c:\users\arjun janamatti\appdata\local\pip\cache\wheels\e7\11\a0\7e65f734d33043735a557b1244569cca327353db9068158076
Successfully built moviepy proglog


Import of required libraries

In [2]:
import speech_recognition as sr 
import moviepy.editor as mp

Conversion from Video to Audio

* I recommend converting it to wav format. It works great with the speech recognition library, which will be covered in the next step.


In [ ]:
clip = mp.VideoFileClip('/content/drive/MyDrive/VID-20201217-WA0001.mp4') 
 
clip.audio.write_audiofile('/content/drive/MyDrive/converted.wav')

In [3]:
r = sr.Recognizer()

In [ ]:
audio_file = sr.AudioFile('/content/drive/MyDrive/converted.wav')

In [ ]:
with audio_file as source:
    audio_file_read = r.listen(source)

result = r.recognize_google(audio_file_read)

In [ ]:
result

### Text Extraction for Large videos

In [4]:
!pip install pydub

In [5]:
from pydub import AudioSegment 
from pydub.silence import split_on_silence 
import os

In [6]:
# create a speech recognition object
r = sr.Recognizer()

# a function that splits the audio file into chunks
# and applies speech recognition

def get_audio_path(video_path):  
    '''
    input the video path to get the audio path as output
    '''
    video_filename = (path_video.split('.')[0]).split('/')[-1]
    b = (path_video.split('.')[0]).split('/')[:-1]
    file_location = '/'.join(b)+'/'
    return file_location+video_filename+'_audio.wav'


def get_large_audio_transcription(path_video, path_audio):
    """
    Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks
    """
    # open the video file 
    clip = mp.VideoFileClip(path_video) 
    #  convert the video file to .wav file
    clip.audio.write_audiofile(path_audio)
    # open the audio file using pydub
    sound = AudioSegment.from_wav(path_audio)  
    # split audio sound where silence is 500 miliseconds or more and get chunks
    chunks = split_on_silence(sound,
        # experiment with this value for your target audio file
        min_silence_len = 500,
        # adjust this per requirement
        silence_thresh = sound.dBFS-14,
        # keep the silence for 1 second, adjustable as well
        keep_silence=500,
    )
    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    # process each chunk 
    for i, audio_chunk in enumerate(chunks, start=1):
        # export audio chunk and save it in
        # the `folder_name` directory.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text += text
    # return the text for all chunks detected
    return whole_text


In [ ]:
path_video = '/content/drive/MyDrive/History of Swear Words Official Trailer Netflix.mp4'
# /content/drive/MyDrive/Video.mp4
result_1 = get_large_audio_transcription(path_video, get_audio_path(path_video))
result_1